In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, datasets
from PIL import Image
import os


transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # centre les valeurs around 0
])

train_dir = 'stegoimagesdataset/train/train/'
val_dir = 'stegoimagesdataset/val/val/'
test_dir = 'stegoimagesdataset/test/test/'

train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
val_dataset = datasets.ImageFolder(root=val_dir, transform=transform)
test_dataset = datasets.ImageFolder(root=test_dir, transform=transform)

batch_size = 64

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print(f"Classes: {train_dataset.classes}")
number_of_classes = len(train_dataset.classes)


Classes: ['clean', 'stego']
Dataset ImageFolder
    Number of datapoints: 16000
    Root location: stegoimagesdataset/train/train/
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )


In [7]:
import torch
import torch.nn as nn
import torchvision.models as models

# Use the new weights parameter instead of pretrained
model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)


# Replace the last layer to match the number of classes in the dataset
model.fc = nn.Linear(model.fc.in_features, number_of_classes)  # 2 classes : clean and stego

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
model = model.to(device)

#print(model)

Using device: cuda


In [ ]:
import torch.optim as optim

def train(model, train_loader, criterion, optimizer, num_epochs=10):

    for epoch in range(num_epochs):
        model.train() # Set the model to training mode
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs in train_loader:
            inputs, labels = inputs[0].to(device), inputs[1].to(device)

            optimizer.zero_grad()
            outputs = model(inputs) # Forward images through the model (Forward pass)
            loss = criterion(outputs, labels) # Calculate loss, moyenne de la perte sur le batch actuel,(perte 1 + perte 2 + ... + perte n) / n
            loss.backward() # Backpropagation of the loss
            optimizer.step() # Update weights

            running_loss += loss.item() * len(inputs) # Donne la somme totale de la perte sur le batch
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).sum().item()
            total += len(labels)

        epoch_loss = running_loss / total # Average loss over the entire dataset
        accuracy = 100 * correct / total # Calculate accuracy
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {accuracy:.2f}%")

    return model

criterion = nn.CrossEntropyLoss() # Cross entropy loss for multi-class classification
optimizer = optim.Adam(model.parameters(), lr=1e-4) # Adam optimizer

trained_model_CNN = train(model, train_loader, criterion, optimizer)

total: 32
total: 64
total: 96
total: 128
total: 160
total: 192
total: 224
total: 256
total: 288
total: 320
total: 352
total: 384


KeyboardInterrupt: 

In [ ]:
import os
import numpy as np
from PIL import Image
from scipy.stats import kurtosis, skew
import pandas as pd

def hjorth_parameters(signal):
    first_deriv = np.diff(signal)
    second_deriv = np.diff(first_deriv)
    var_zero = np.var(signal)
    var_d1 = np.var(first_deriv)
    var_d2 = np.var(second_deriv)
    mobility = np.sqrt(var_d1 / var_zero)
    complexity = np.sqrt((var_d2 / var_d1) - (var_d1 / var_zero))
    return mobility, complexity

def extract_features_from_image(img_path):
    img = Image.open(img_path).convert('L')  # grayscale
    data = np.asarray(img).flatten()
    std = np.std(data)
    range_val = np.max(data) - np.min(data)
    median = np.median(data)
    geo_median = np.exp(np.mean(np.log(data + 1e-5)))
    skewness = skew(data)
    kurt = kurtosis(data)
    hj_mob, hj_comp = hjorth_parameters(data)
    return [std, range_val, median, geo_median, skewness, kurt, hj_mob, hj_comp]

def build_stat_feature_dataset(root_path):
    rows = []
    for label_name in os.listdir(root_path):  # normal, stego
        class_path = os.path.join(root_path, label_name)
        label = 0 if label_name.lower() == 'clean' else 1
        for img_name in os.listdir(class_path):
            img_path = os.path.join(class_path, img_name)
            features = extract_features_from_image(img_path)
            rows.append(features + [label])
    df = pd.DataFrame(rows, columns=[
        'std', 'range', 'median', 'geo_median',
        'skewness', 'kurtosis', 'hj_mob', 'hj_comp', 'label'
    ])
    return df

df = build_stat_feature_dataset(train_dir)

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models

class CNNStatFusion(nn.Module):
    def __init__(self):
        super().__init__()
        base_model = models.resnet18(pretrained=True)
        self.cnn = nn.Sequential(*list(base_model.children())[:-1])
        self.stat_fc = nn.Sequential(
            nn.Linear(8, 64),
            nn.ReLU(),
            nn.Linear(64, 64)
        )
        self.final_fc = nn.Sequential(
            nn.Linear(512 + 64, 128),
            nn.ReLU(),
            nn.Linear(128, 2)
        )

    def forward(self, image, stat_feats):
        cnn_feat = self.cnn(image).squeeze()
        stat_feat = self.stat_fc(stat_feats)
        fusion = torch.cat([cnn_feat, stat_feat], dim=1)
        return self.final_fc(fusion)


In [ ]:
# validation step
def validate(model, val_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
        

    accuracy = 100 * correct / total
    return running_loss / len(val_loader), accuracy

# Validation loop
val_loss, val_accuracy = validate(trained_model_CNN, val_loader, criterion, device)
print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%")



Validation Loss: 0.4117, Validation Accuracy: 75.00%


In [ ]:
# Save the model
#torch.save(trained_model_CNN.state_dict(), 'model.pth')
# Load the model
model.load_state_dict(torch.load('model.pth'))
# Test the model    
def test(model, test_loader, device):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy

# Test the model
test_accuracy = test(model, test_loader, device)
print(f"Test Accuracy: {test_accuracy:.2f}%")

C:\Users\kilio\AppData\Local\Temp\ipykernel_18216\3423573168.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('model.pth'))


Test Accuracy: 30.00%
